In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [27]:

class BinaryClassifier(nn.Module):
    def __init__(self, input_dim, hidden_layers):
        super(BinaryClassifier, self).__init__()
        layers = []
        #add fully connected layers with the input dim dividing by two each time the input shape
        output_dim = max(input_dim//(2), 1)
        for i in range(hidden_layers):
            i += 1
            layers.append(nn.Linear(input_dim,output_dim))
            layers.append(nn.ReLU())
            input_dim = output_dim
            output_dim = max(input_dim//(2*i), 1)
        #add final layer with sigmiod output and final linear layer
        layers.append(nn.Linear(input_dim,1))
        layers.append(nn.Sigmoid())

        #create a Seqeuntial implementation, no additional layers but provides one line to run through all layers
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        #apply the neural net
        return self.network(x)

In [29]:
#input shape is not correct
input_shape = 9
hidden_layers = 5
model_1 = BinaryClassifier(input_shape, hidden_layers)

In [ ]:
loss_fn = torch.nn.BCELoss()